In [1]:
from random import randrange, choice
from statistics import mean, stdev
from copy import deepcopy

In [2]:
class State:
    def __init__(self, no_of_queens, board_size, queen_pos=None):
        self.board_size = board_size
        if queen_pos is None:
            self.no_of_queens = no_of_queens
            self.queen_pos = frozenset(self.random_queen_position())
        else:
            self.queen_pos = frozenset(queen_pos)
            self.no_of_queens = len(self.queen_pos)

    # Place N queens at random position on the board
    # Returns list of tuples with the queen coordinates
    def random_queen_position(self):
        open_columns = list(range(self.board_size))
        queen_pos = [(open_columns.pop(randrange(len(open_columns))), randrange(self.board_size)) for _ in
                           range(self.no_of_queens)]
        return queen_pos

    # Returns all the possible moves of the current state
    def possible_moves(self):
        moves = []
        parent_queen_pos = list(self.queen_pos)
        for queen_index, queen in enumerate(parent_queen_pos):
            new_positions = [(queen[0], row) for row in range(self.board_size) if row != queen[1]]
            for new_position in new_positions:
                queen_pos = deepcopy(parent_queen_pos)
                queen_pos[queen_index] = new_position
                moves.append(State(self.no_of_queens, self.board_size, queen_pos))
        return moves

    # Checks if 2 queens are attacking each other or not
    def is_attacking(self, queens, a, b):
        if (a[0] == b[0]) or (a[1] == b[1]) or (abs(a[0] - b[0]) == abs(a[1] - b[1])):
            return True
        else:
            return False
            
    # Returns number of attacking pairs of queens
    def heuristic_value(self):
        attacking_pairs = []
        queen_pos = list(self.queen_pos)
        left_to_check = deepcopy(queen_pos)
        while left_to_check:
            a = left_to_check.pop()
            for b in left_to_check:
                if self.is_attacking(queen_pos, a, b):
                    attacking_pairs.append([a, b])
        return len(attacking_pairs)

In [3]:
# N queen problem
class NQueenProblem:
    def __init__(self, N, start_state=None):
        if not start_state:
            start_state = State(no_of_queens=N, board_size=N)
        self.start_state = start_state

    # Check if it is a goal state or not
    def is_goal_state(self, state):
        return state.heuristic_value() == 0

    # Calculating heuristic value
    def heuristic(self, state):
        return state.heuristic_value()

In [4]:
def min_heuristic_move(current_state, problem):
    moves = current_state.possible_moves()
    moves_heuristic = [problem.heuristic(move) for move in moves]
    min_heuristic = min(moves_heuristic)

    best_move = choice([move for index, move in enumerate(moves) if moves_heuristic[
        index] == min_heuristic])
    return best_move

# Steepest ascent hill climbing algorithm 
def steepest_ascent_hill_climb(problem):
    current_state = problem.start_state
    current_state_heuristic = problem.heuristic(current_state)
    path = []

    while True:
        path.append(current_state)
        best_move = min_heuristic_move(current_state, problem)
        best_move_heuristic = problem.heuristic(best_move)

        if best_move_heuristic >= current_state_heuristic:
            break
        else:
            current_state = best_move
            current_state_heuristic = best_move_heuristic

    return {'result': 'success' if problem.is_goal_state(current_state) else 'failure',
            'solution': path,
            'problem': problem}

In [5]:
def random_heuristic_move(current_state, problem):
    moves = current_state.possible_moves()
    moves_heuristic = [problem.heuristic(move) for move in moves]
    current_heuristic = problem.heuristic(current_state)

    better_sol = []
    for index, move in enumerate(moves):
        if moves_heuristic[index] < current_heuristic:
            better_sol.append(move)
    if len(better_sol) != 0:
        return choice(better_sol)
    else:
        return current_state

# Stochastic hill climbing algorithm
def stochastic_hill_climb(problem):
    current_state = problem.start_state
    current_state_heuristic = problem.heuristic(current_state)
    path = []

    while True:
        path.append(current_state)
        random_move = random_heuristic_move(current_state, problem)
        random_move_heuristic = problem.heuristic(random_move)

        if random_move_heuristic >= current_state_heuristic:
            break
        else:
            current_state = random_move
            current_state_heuristic = random_move_heuristic

    return {'result': 'success' if problem.is_goal_state(current_state) else 'failure',
            'solution': path,
            'problem': problem}

In [6]:
# Returns a results list containing all the possible solutions, successes and failures
def print_result(problem_set, search_function):
    results = []
    for problem in problem_set:
        result = search_function(problem)
        result['path_length'] = len(result['solution']) - 1
        results.append(result)
    results = [results,
               [result for result in results if result['result'] == 'success'],
               [result for result in results if result['result'] == 'failure']]

    num_iterations = len(results[0])
    successful_steps = []
    failure_steps = []
    
    for x in results[1]:
        successful_steps.append(x['path_length'])
    for x in results[2]:
        failure_steps.append(x['path_length'])
        
    print("Success : ", len(results[1])*100/num_iterations, "%")
    print("No of steps in each successful run : ", successful_steps)
    print("Average no of steps when success : ", round(mean(successful_steps)), "\n")
    print("Failure : ", len(results[2])*100/num_iterations, "%")
    print("No of steps in each failed run : ", failure_steps)
    print("Average no of steps when failed : ", round(mean(failure_steps)))

In [7]:
print("Steepest Ascent Hill Climbing for 1000 randomly generated 8 Queens problem ")
queens_state = [NQueenProblem(8) for _ in range(1000)]
print_result(queens_state, steepest_ascent_hill_climb)

Steepest Ascent Hill Climbing for 1000 randomly generated 8 Queens problem 
Success :  17.1 %
No of steps in each successful run :  [3, 4, 3, 5, 5, 3, 3, 2, 3, 3, 4, 5, 5, 5, 4, 5, 3, 4, 3, 2, 4, 3, 4, 3, 5, 3, 5, 3, 3, 4, 3, 4, 4, 3, 4, 4, 4, 3, 4, 5, 3, 4, 3, 4, 4, 6, 4, 3, 4, 5, 5, 4, 3, 5, 5, 3, 5, 4, 5, 3, 4, 3, 3, 4, 3, 5, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 6, 3, 5, 5, 4, 5, 3, 4, 6, 4, 2, 4, 4, 4, 5, 4, 4, 6, 4, 3, 2, 3, 4, 5, 4, 5, 4, 3, 5, 3, 5, 5, 3, 4, 5, 4, 4, 3, 3, 3, 5, 5, 4, 5, 4, 5, 5, 4, 4, 4, 4, 4, 3, 5, 4, 6, 5, 4, 6, 4, 3, 5, 4, 4, 4, 5, 4, 4, 5, 2, 4, 5, 4, 4, 3, 4, 4, 6, 5, 5, 4, 5, 5, 4, 3, 5, 5, 5, 4, 5, 5, 4, 4, 4]
Average no of steps when success :  4 

Failure :  82.9 %
No of steps in each failed run :  [1, 4, 3, 4, 4, 5, 3, 2, 3, 4, 4, 4, 2, 3, 3, 3, 4, 3, 2, 3, 3, 4, 3, 3, 3, 3, 3, 4, 3, 3, 2, 2, 4, 3, 3, 3, 5, 2, 3, 3, 2, 3, 5, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 4, 3, 3, 2, 2, 2, 2, 2, 3, 3, 3, 4, 2, 2, 4, 3, 4, 4, 4, 3, 2, 3, 2, 3, 2, 4, 3, 3, 2, 2, 1, 4, 

In [8]:
print("Steepest Ascent Hill Climbing for 1000 randomly generated 10 Queens problem ")
queens_state = [NQueenProblem(10) for _ in range(1000)]
print_result(queens_state, steepest_ascent_hill_climb)

Steepest Ascent Hill Climbing for 1000 randomly generated 10 Queens problem 
Success :  5.7 %
No of steps in each successful run :  [7, 7, 5, 6, 6, 6, 4, 5, 7, 5, 6, 5, 4, 3, 5, 6, 5, 4, 8, 5, 6, 6, 5, 6, 6, 3, 4, 4, 7, 3, 6, 7, 5, 5, 4, 7, 5, 6, 5, 5, 5, 4, 5, 5, 5, 7, 6, 6, 6, 3, 5, 7, 4, 5, 6, 5, 6]
Average no of steps when success :  5 

Failure :  94.3 %
No of steps in each failed run :  [5, 4, 5, 4, 4, 3, 3, 5, 4, 5, 4, 6, 4, 4, 4, 4, 4, 3, 5, 3, 5, 5, 4, 4, 4, 4, 4, 5, 3, 1, 5, 5, 5, 3, 2, 3, 4, 5, 4, 6, 3, 4, 3, 4, 3, 3, 5, 5, 3, 5, 3, 5, 2, 5, 4, 3, 3, 4, 4, 2, 4, 5, 2, 4, 4, 3, 4, 4, 2, 3, 5, 3, 2, 5, 5, 3, 3, 3, 5, 3, 3, 4, 5, 4, 3, 3, 4, 4, 4, 4, 1, 5, 6, 3, 3, 3, 5, 3, 4, 5, 4, 4, 5, 3, 4, 5, 5, 7, 2, 3, 3, 2, 3, 3, 5, 4, 5, 3, 3, 4, 4, 4, 3, 2, 4, 5, 6, 4, 4, 4, 5, 2, 4, 4, 5, 3, 3, 2, 4, 4, 3, 5, 5, 5, 3, 3, 6, 4, 4, 4, 3, 3, 4, 5, 3, 5, 3, 3, 4, 2, 4, 4, 3, 5, 4, 4, 6, 3, 4, 2, 3, 4, 3, 4, 4, 3, 4, 3, 5, 4, 3, 5, 4, 4, 4, 4, 4, 4, 4, 5, 4, 5, 5, 4, 4, 5, 3, 5, 3, 4, 5, 

In [9]:
print("Stochastic Hill Climbing for 1000 randomly generated 8 Queens problem ")
queens_state = [NQueenProblem(8) for _ in range(1000)]
print_result(queens_state, stochastic_hill_climb)

Stochastic Hill Climbing for 1000 randomly generated 8 Queens problem 
Success :  16.2 %
No of steps in each successful run :  [4, 8, 2, 7, 5, 5, 7, 4, 2, 8, 8, 7, 5, 5, 6, 6, 6, 5, 6, 4, 7, 4, 7, 5, 6, 9, 5, 4, 5, 6, 5, 9, 9, 6, 6, 4, 5, 5, 5, 8, 4, 6, 4, 5, 6, 4, 7, 5, 5, 7, 6, 7, 6, 7, 2, 9, 5, 6, 7, 2, 6, 9, 3, 6, 6, 8, 9, 8, 5, 6, 5, 8, 7, 6, 9, 9, 6, 8, 4, 5, 5, 8, 4, 6, 4, 4, 5, 9, 6, 5, 6, 4, 7, 6, 6, 7, 5, 6, 4, 4, 4, 7, 4, 4, 7, 7, 7, 2, 5, 6, 7, 9, 7, 5, 3, 8, 7, 9, 8, 8, 7, 7, 5, 7, 6, 7, 6, 7, 5, 4, 8, 5, 5, 7, 6, 8, 5, 4, 6, 5, 3, 8, 3, 5, 8, 5, 7, 5, 6, 8, 4, 4, 10, 7, 5, 7, 6, 4, 5, 8, 4, 5]
Average no of steps when success :  6 

Failure :  83.8 %
No of steps in each failed run :  [5, 7, 4, 5, 5, 7, 6, 1, 6, 5, 5, 6, 4, 4, 4, 8, 6, 5, 1, 5, 6, 6, 4, 3, 7, 5, 6, 3, 4, 4, 4, 3, 5, 2, 6, 5, 6, 3, 6, 5, 4, 4, 5, 3, 4, 6, 7, 6, 5, 5, 3, 7, 5, 6, 4, 4, 2, 8, 7, 4, 7, 4, 1, 5, 4, 5, 5, 6, 5, 6, 4, 4, 6, 3, 4, 5, 5, 5, 5, 7, 4, 2, 9, 4, 3, 2, 7, 5, 6, 6, 6, 4, 4, 9, 2, 7, 3, 5

In [10]:
print("Steepest Ascent Hill Climbing for 1000 randomly generated 20 Queens problem ")
queens_state = [NQueenProblem(20) for _ in range(1000)]
print_result(queens_state, steepest_ascent_hill_climb)

Steepest Ascent Hill Climbing for 1000 randomly generated 20 Queens problem 
Success :  2.0 %
No of steps in each successful run :  [12, 11, 10, 12, 11, 10, 12, 10, 10, 12, 10, 12, 12, 13, 10, 10, 11, 10, 11, 10]
Average no of steps when success :  11 

Failure :  98.0 %
No of steps in each failed run :  [7, 8, 9, 8, 7, 9, 8, 9, 9, 9, 7, 11, 11, 8, 7, 6, 8, 9, 11, 8, 7, 10, 6, 8, 10, 9, 10, 8, 8, 7, 6, 8, 6, 11, 10, 7, 7, 6, 7, 8, 9, 7, 10, 9, 7, 10, 8, 10, 10, 8, 6, 9, 8, 10, 7, 7, 6, 8, 11, 8, 8, 10, 8, 8, 9, 7, 7, 6, 8, 8, 10, 7, 10, 9, 6, 9, 6, 6, 8, 8, 8, 8, 9, 12, 10, 10, 11, 8, 8, 9, 8, 7, 9, 6, 7, 9, 6, 10, 8, 8, 11, 7, 8, 8, 8, 8, 7, 7, 9, 9, 10, 9, 6, 9, 7, 12, 8, 6, 8, 6, 8, 10, 8, 10, 11, 8, 8, 7, 9, 5, 10, 6, 8, 10, 8, 10, 9, 9, 9, 12, 9, 11, 6, 6, 6, 8, 10, 6, 10, 9, 6, 8, 7, 9, 7, 7, 8, 9, 7, 8, 6, 6, 8, 8, 8, 8, 9, 8, 7, 8, 10, 9, 9, 8, 11, 10, 8, 6, 8, 9, 8, 9, 7, 9, 8, 7, 8, 9, 5, 11, 7, 9, 7, 8, 9, 8, 10, 8, 10, 7, 9, 8, 11, 9, 10, 8, 8, 9, 9, 6, 8, 8, 6, 8, 6, 9, 7,